# User 144

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

# Carregar o dataframe inicial
User = pd.read_excel('EVIO_history_01-02-2023_29-02-2024.xlsx')
User = User[['Start date','Stop date','Duration (min)', 'Total Energy (kWh)', 'Nº cartão EVIO']]

# Remover valores de carregamento de energia inferiores a 1 kWh
User = User[User['Total Energy (kWh)'] >= 1]

User = User[User['Nº cartão EVIO'] == 144]

# Remover valores de duração de carregamento inferiores a 5 minutos
User = User[User['Duration (min)'] >= 5]

User = User[['Start date','Stop date','Duration (min)', 'Total Energy (kWh)']]

print(User)

              Start date           Stop date  Duration (min)  \
1     02/29/2024 | 10:44  02/29/2024 | 11:24          39.483   
15    02/27/2024 | 16:35  02/27/2024 | 17:07          32.583   
37    02/23/2024 | 08:24  02/23/2024 | 09:49          85.183   
48    02/21/2024 | 13:14  02/21/2024 | 14:06          51.883   
78    02/14/2024 | 10:18  02/14/2024 | 11:05          46.283   
...                  ...                 ...             ...   
1603  03/17/2023 | 14:07  03/17/2023 | 15:39          92.517   
1629  03/10/2023 | 08:42  03/10/2023 | 11:24         161.967   
1639  03/08/2023 | 14:03  03/08/2023 | 17:14         190.867   
1651  03/06/2023 | 13:35  03/06/2023 | 17:46         250.533   
1662  03/03/2023 | 21:59  03/03/2023 | 22:52          52.567   

      Total Energy (kWh)  
1                  19.95  
15                 29.61  
37                 14.47  
48                 41.44  
78                 49.60  
...                  ...  
1603               15.51  
1629           

In [2]:
# Convertendo as colunas de data para o formato de data especificado
User['Start date'] = pd.to_datetime(User['Start date'], format='%m/%d/%Y | %H:%M')
User['Stop date'] = pd.to_datetime(User['Stop date'], format='%m/%d/%Y | %H:%M')


# Criando uma nova coluna 'Date' que contém apenas a data (sem a hora)
User['Date'] = User['Start date'].dt.date

# Convertendo os valores de energia para o formato numérico adequado
#User['Total Energy (kWh)'] = User['Total Energy (kWh)'].str.replace(',', '.').astype(float)

# Agrupando os dados pela coluna 'Date' e somando os valores de 'Total Energy (kWh)'
User = User.groupby('Date')['Total Energy (kWh)'].sum().reset_index()

print(User)

# Criando um intervalo de datas que cubra todo o período de interesse
date_range = pd.date_range(start=User['Date'].min(), end=User['Date'].max(), freq='D')

# Criando um DataFrame com o intervalo de datas
date_df = pd.DataFrame({'Date': date_range})

# Renomeando a coluna 'Date' para 'Date_new'
date_df.rename(columns={'Date': 'Date_new'}, inplace=True)

# Convertendo a coluna 'Date' em 'daily_energy_sum' para datetime
User['Date'] = pd.to_datetime(User['Date'])

# Mesclando os DataFrames usando merge
merged_df = pd.merge(date_df, User, left_on='Date_new', right_on='Date', how='left')

# Preenchendo os valores ausentes na coluna 'Total Energy (kWh)' com zero
merged_df['Total Energy (kWh)'].fillna(0, inplace=True)
merged_df = merged_df.drop(columns=['Date'])
merged_df.rename(columns={'Date_new': 'Date'}, inplace=True)

print(merged_df)

          Date  Total Energy (kWh)
0   2023-03-03               46.92
1   2023-03-06               40.94
2   2023-03-08               32.80
3   2023-03-10               27.92
4   2023-03-17               15.51
..         ...                 ...
93  2024-02-14               49.60
94  2024-02-21               41.44
95  2024-02-23               14.47
96  2024-02-27               29.61
97  2024-02-29               19.95

[98 rows x 2 columns]
          Date  Total Energy (kWh)
0   2023-03-03               46.92
1   2023-03-04                0.00
2   2023-03-05                0.00
3   2023-03-06               40.94
4   2023-03-07                0.00
..         ...                 ...
359 2024-02-25                0.00
360 2024-02-26                0.00
361 2024-02-27               29.61
362 2024-02-28                0.00
363 2024-02-29               19.95

[364 rows x 2 columns]


In [3]:
merged_df['Weekday'] = merged_df['Date'].dt.day_name()

weekday_mapping = {
    'Monday': 1,
    'Tuesday': 2,
    'Wednesday': 3,
    'Thursday': 4,
    'Friday': 5,
    'Saturday': 6,
    'Sunday': 7
}

merged_df['Weekday'] = merged_df['Weekday'].map(weekday_mapping)

print(merged_df)

          Date  Total Energy (kWh)  Weekday
0   2023-03-03               46.92        5
1   2023-03-04                0.00        6
2   2023-03-05                0.00        7
3   2023-03-06               40.94        1
4   2023-03-07                0.00        2
..         ...                 ...      ...
359 2024-02-25                0.00        7
360 2024-02-26                0.00        1
361 2024-02-27               29.61        2
362 2024-02-28                0.00        3
363 2024-02-29               19.95        4

[364 rows x 3 columns]


In [4]:
# Adicionar a coluna 'Carregar'
merged_df['Carregar'] = merged_df['Total Energy (kWh)'] > 0

# Visualizar as primeiras linhas do DataFrame
print(merged_df)

# Criar uma coluna 'Last_Charge_Date' que desloca a data do último carregamento
merged_df['Last_Charge_Date'] = merged_df['Date'].where(merged_df['Carregar']).ffill()

# Calcular os dias desde o último carregamento
merged_df['Days_since_last_charge'] = (merged_df['Date'] - merged_df['Last_Charge_Date']).dt.days

# Preencher valores NaN em 'Days_since_last_charge' com 0 (caso não tenha ocorrido carregamento antes)
merged_df['Days_since_last_charge'] = merged_df['Days_since_last_charge'].fillna(0).astype(int)

# Visualizar o DataFrame com a nova coluna
print(merged_df)

          Date  Total Energy (kWh)  Weekday  Carregar
0   2023-03-03               46.92        5      True
1   2023-03-04                0.00        6     False
2   2023-03-05                0.00        7     False
3   2023-03-06               40.94        1      True
4   2023-03-07                0.00        2     False
..         ...                 ...      ...       ...
359 2024-02-25                0.00        7     False
360 2024-02-26                0.00        1     False
361 2024-02-27               29.61        2      True
362 2024-02-28                0.00        3     False
363 2024-02-29               19.95        4      True

[364 rows x 4 columns]
          Date  Total Energy (kWh)  Weekday  Carregar Last_Charge_Date  \
0   2023-03-03               46.92        5      True       2023-03-03   
1   2023-03-04                0.00        6     False       2023-03-03   
2   2023-03-05                0.00        7     False       2023-03-03   
3   2023-03-06               40.

In [5]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

# Supondo que merged_df é o DataFrame com os dados fornecidos e já possui as colunas necessárias
# Adicionar uma coluna 'Carregar' baseada em 'Total Energy (kWh)'
merged_df['Carregar'] = (merged_df['Total Energy (kWh)'] > 0).astype(int)

# Features e rótulo para a previsão binária
X = merged_df[['Weekday', 'Days_since_last_charge']].values
y = merged_df['Carregar'].astype(int).values

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

# Treinar o modelo de regressão logística
logistic_model = LogisticRegression()
logistic_model.fit(X_train, y_train)

# Fazer previsões para o conjunto de teste
predictions = logistic_model.predict(X_test)

# Avaliar a precisão do modelo
accuracy = accuracy_score(y_test, predictions)
print("Acurácia do modelo:", accuracy)

# Matriz de confusão
conf_matrix = confusion_matrix(y_test, predictions)
print("Matriz de confusão:")
print(conf_matrix)

# Filtrar dados onde houve carregamento
charging_data = merged_df[merged_df['Carregar'] == 1]

# Features para o modelo de regressão
X_reg = charging_data[['Weekday', 'Days_since_last_charge']].values
y_reg = charging_data['Total Energy (kWh)'].values

# Treinar o modelo de regressão linear
reg_model = LinearRegression()
reg_model.fit(X_reg, y_reg)

# Fazer previsões (usando X_test completo)
energy_predictions = np.zeros(len(X_test))  # Inicializar com zeros
charging_indices = [i for i, pred in enumerate(predictions) if pred == 1]

if len(charging_indices) > 0:
    X_test_charging = X_test[charging_indices]
    energy_predictions_charging = reg_model.predict(X_test_charging)
    for idx, val in zip(charging_indices, energy_predictions_charging):
        energy_predictions[idx] = val

# Consolidação dos resultados com datas corretas
test_indices = X_test[:, 0].astype(int)
result_df = pd.DataFrame({
    'Date': merged_df.iloc[test_indices].index,
    'Weekday': X_test[:, 0],
    'Days_since_last_charge': X_test[:, 1],
    'Carregar (Previsão)': predictions,
    'Total Energy (Previsão)': energy_predictions
})

result_df['Carregar (Previsão)'] = result_df['Carregar (Previsão)'].map({0: 'Não Carregar', 1: 'Carregar'})

print("Previsões finais:")
print(result_df)

Acurácia do modelo: 1.0
Matriz de confusão:
[[55  0]
 [ 0 18]]
Previsões finais:
    Date  Weekday  Days_since_last_charge Carregar (Previsão)  \
0      2        2                       0            Carregar   
1      3        3                       2        Não Carregar   
2      6        6                       1        Não Carregar   
3      2        2                       7        Não Carregar   
4      6        6                       2        Não Carregar   
..   ...      ...                     ...                 ...   
68     4        4                       5        Não Carregar   
69     3        3                       2        Não Carregar   
70     1        1                       3        Não Carregar   
71     1        1                       4        Não Carregar   
72     2        2                       0            Carregar   

    Total Energy (Previsão)  
0                  35.76026  
1                   0.00000  
2                   0.00000  
3                 

In [6]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import GradientBoostingRegressor

# Supondo que merged_df é o DataFrame com os dados fornecidos e já possui as colunas necessárias
# Adicionar uma coluna 'Carregar' baseada em 'Total Energy (kWh)'
merged_df['Carregar'] = (merged_df['Total Energy (kWh)'] > 0).astype(int)

# Features e rótulo para a previsão binária
X = merged_df[['Weekday', 'Days_since_last_charge']].values
y = merged_df['Carregar'].astype(int).values

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

# Treinar o modelo de regressão logística
logistic_model = LogisticRegression()
logistic_model.fit(X_train, y_train)

# Fazer previsões para o conjunto de teste
predictions = logistic_model.predict(X_test)

# Avaliar a precisão do modelo
accuracy = accuracy_score(y_test, predictions)
print("Acurácia do modelo:", accuracy)

# Matriz de confusão
conf_matrix = confusion_matrix(y_test, predictions)
print("Matriz de confusão:")
print(conf_matrix)

# Filtrar dados onde houve carregamento
charging_data = merged_df[merged_df['Carregar'] == 1]

# Features para o modelo de regressão
X_reg = charging_data[['Weekday', 'Days_since_last_charge']].values
y_reg = charging_data['Total Energy (kWh)'].values

# Treinar o modelo de regressão linear
reg_model = GradientBoostingRegressor()
reg_model.fit(X_reg, y_reg)

# Fazer previsões (usando X_test completo)
energy_predictions = np.zeros(len(X_test))  # Inicializar com zeros
charging_indices = [i for i, pred in enumerate(predictions) if pred == 1]

if len(charging_indices) > 0:
    X_test_charging = X_test[charging_indices]
    energy_predictions_charging = reg_model.predict(X_test_charging)
    for idx, val in zip(charging_indices, energy_predictions_charging):
        energy_predictions[idx] = val

# Consolidação dos resultados com datas corretas
test_indices = X_test[:, 0].astype(int)
result_df = pd.DataFrame({
    'Date': merged_df.iloc[test_indices].index,
    'Weekday': X_test[:, 0],
    'Days_since_last_charge': X_test[:, 1],
    'Carregar (Previsão)': predictions,
    'Total Energy (Previsão)': energy_predictions
})

result_df['Carregar (Previsão)'] = result_df['Carregar (Previsão)'].map({0: 'Não Carregar', 1: 'Carregar'})

print("Previsões finais:")
print(result_df)

Acurácia do modelo: 1.0
Matriz de confusão:
[[55  0]
 [ 0 18]]
Previsões finais:
    Date  Weekday  Days_since_last_charge Carregar (Previsão)  \
0      2        2                       0            Carregar   
1      3        3                       2        Não Carregar   
2      6        6                       1        Não Carregar   
3      2        2                       7        Não Carregar   
4      6        6                       2        Não Carregar   
..   ...      ...                     ...                 ...   
68     4        4                       5        Não Carregar   
69     3        3                       2        Não Carregar   
70     1        1                       3        Não Carregar   
71     1        1                       4        Não Carregar   
72     2        2                       0            Carregar   

    Total Energy (Previsão)  
0                 34.927525  
1                  0.000000  
2                  0.000000  
3                 

# User 44

In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

# Carregar o dataframe inicial
User = pd.read_excel('EVIO_history_01-02-2023_29-02-2024.xlsx')
User = User[['Start date','Stop date','Duration (min)', 'Total Energy (kWh)', 'Nº cartão EVIO']]

# Remover valores de carregamento de energia inferiores a 1 kWh
User = User[User['Total Energy (kWh)'] >= 1]

User = User[User['Nº cartão EVIO'] == 46]

# Remover valores de duração de carregamento inferiores a 5 minutos
User = User[User['Duration (min)'] >= 5]

User = User[['Start date','Stop date','Duration (min)', 'Total Energy (kWh)']]

print(User)

              Start date           Stop date  Duration (min)  \
5     02/29/2024 | 08:57  02/29/2024 | 10:11          74.783   
10    02/28/2024 | 08:49  02/28/2024 | 12:20         211.183   
20    02/27/2024 | 08:50  02/27/2024 | 10:24          93.817   
44    02/22/2024 | 08:43  02/22/2024 | 12:42         239.583   
70    02/16/2024 | 08:39  02/16/2024 | 13:35         295.633   
...                  ...                 ...             ...   
1648  03/07/2023 | 08:36  03/07/2023 | 09:48          71.733   
1667  03/03/2023 | 08:35  03/03/2023 | 09:52          77.333   
1691  02/23/2023 | 08:47  02/23/2023 | 10:36         109.450   
1718  02/14/2023 | 13:20  02/14/2023 | 17:41         260.083   
1723  02/14/2023 | 08:40  02/14/2023 | 10:00          79.867   

      Total Energy (kWh)  
5                   9.14  
10                  8.60  
20                  9.57  
44                 10.84  
70                 10.76  
...                  ...  
1648                8.20  
1667           

In [14]:
# Convertendo as colunas de data para o formato de data especificado
User['Start date'] = pd.to_datetime(User['Start date'], format='%m/%d/%Y | %H:%M')
User['Stop date'] = pd.to_datetime(User['Stop date'], format='%m/%d/%Y | %H:%M')


# Criando uma nova coluna 'Date' que contém apenas a data (sem a hora)
User['Date'] = User['Start date'].dt.date

# Convertendo os valores de energia para o formato numérico adequado
#User['Total Energy (kWh)'] = User['Total Energy (kWh)'].str.replace(',', '.').astype(float)

# Agrupando os dados pela coluna 'Date' e somando os valores de 'Total Energy (kWh)'
User = User.groupby('Date')['Total Energy (kWh)'].sum().reset_index()

print(User)

# Criando um intervalo de datas que cubra todo o período de interesse
date_range = pd.date_range(start=User['Date'].min(), end=User['Date'].max(), freq='D')

# Criando um DataFrame com o intervalo de datas
date_df = pd.DataFrame({'Date': date_range})

# Renomeando a coluna 'Date' para 'Date_new'
date_df.rename(columns={'Date': 'Date_new'}, inplace=True)

# Convertendo a coluna 'Date' em 'daily_energy_sum' para datetime
User['Date'] = pd.to_datetime(User['Date'])

# Mesclando os DataFrames usando merge
merged_df = pd.merge(date_df, User, left_on='Date_new', right_on='Date', how='left')

# Preenchendo os valores ausentes na coluna 'Total Energy (kWh)' com zero
merged_df['Total Energy (kWh)'].fillna(0, inplace=True)
merged_df = merged_df.drop(columns=['Date'])
merged_df.rename(columns={'Date_new': 'Date'}, inplace=True)

print(merged_df)

           Date  Total Energy (kWh)
0    2023-02-14               14.10
1    2023-02-23               10.69
2    2023-03-03                8.74
3    2023-03-07                8.20
4    2023-03-20               10.73
..          ...                 ...
106  2024-02-16               10.76
107  2024-02-22               10.84
108  2024-02-27                9.57
109  2024-02-28                8.60
110  2024-02-29                9.14

[111 rows x 2 columns]
          Date  Total Energy (kWh)
0   2023-02-14               14.10
1   2023-02-15                0.00
2   2023-02-16                0.00
3   2023-02-17                0.00
4   2023-02-18                0.00
..         ...                 ...
376 2024-02-25                0.00
377 2024-02-26                0.00
378 2024-02-27                9.57
379 2024-02-28                8.60
380 2024-02-29                9.14

[381 rows x 2 columns]


In [15]:
merged_df['Weekday'] = merged_df['Date'].dt.day_name()

weekday_mapping = {
    'Monday': 1,
    'Tuesday': 2,
    'Wednesday': 3,
    'Thursday': 4,
    'Friday': 5,
    'Saturday': 6,
    'Sunday': 7
}

merged_df['Weekday'] = merged_df['Weekday'].map(weekday_mapping)

print(merged_df)

          Date  Total Energy (kWh)  Weekday
0   2023-02-14               14.10        2
1   2023-02-15                0.00        3
2   2023-02-16                0.00        4
3   2023-02-17                0.00        5
4   2023-02-18                0.00        6
..         ...                 ...      ...
376 2024-02-25                0.00        7
377 2024-02-26                0.00        1
378 2024-02-27                9.57        2
379 2024-02-28                8.60        3
380 2024-02-29                9.14        4

[381 rows x 3 columns]


In [16]:
# Adicionar a coluna 'Carregar'
merged_df['Carregar'] = merged_df['Total Energy (kWh)'] > 0

# Visualizar as primeiras linhas do DataFrame
print(merged_df)

# Criar uma coluna 'Last_Charge_Date' que desloca a data do último carregamento
merged_df['Last_Charge_Date'] = merged_df['Date'].where(merged_df['Carregar']).ffill()

# Calcular os dias desde o último carregamento
merged_df['Days_since_last_charge'] = (merged_df['Date'] - merged_df['Last_Charge_Date']).dt.days

# Preencher valores NaN em 'Days_since_last_charge' com 0 (caso não tenha ocorrido carregamento antes)
merged_df['Days_since_last_charge'] = merged_df['Days_since_last_charge'].fillna(0).astype(int)

# Visualizar o DataFrame com a nova coluna
print(merged_df)

          Date  Total Energy (kWh)  Weekday  Carregar
0   2023-02-14               14.10        2      True
1   2023-02-15                0.00        3     False
2   2023-02-16                0.00        4     False
3   2023-02-17                0.00        5     False
4   2023-02-18                0.00        6     False
..         ...                 ...      ...       ...
376 2024-02-25                0.00        7     False
377 2024-02-26                0.00        1     False
378 2024-02-27                9.57        2      True
379 2024-02-28                8.60        3      True
380 2024-02-29                9.14        4      True

[381 rows x 4 columns]
          Date  Total Energy (kWh)  Weekday  Carregar Last_Charge_Date  \
0   2023-02-14               14.10        2      True       2023-02-14   
1   2023-02-15                0.00        3     False       2023-02-14   
2   2023-02-16                0.00        4     False       2023-02-14   
3   2023-02-17                0.

In [17]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import GradientBoostingRegressor

# Supondo que merged_df é o DataFrame com os dados fornecidos e já possui as colunas necessárias
# Adicionar uma coluna 'Carregar' baseada em 'Total Energy (kWh)'
merged_df['Carregar'] = (merged_df['Total Energy (kWh)'] > 0).astype(int)

# Features e rótulo para a previsão binária
X = merged_df[['Weekday', 'Days_since_last_charge']].values
y = merged_df['Carregar'].astype(int).values

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

# Treinar o modelo de regressão logística
logistic_model = LogisticRegression()
logistic_model.fit(X_train, y_train)

# Fazer previsões para o conjunto de teste
predictions = logistic_model.predict(X_test)

# Avaliar a precisão do modelo
accuracy = accuracy_score(y_test, predictions)
print("Acurácia do modelo:", accuracy)

# Matriz de confusão
conf_matrix = confusion_matrix(y_test, predictions)
print("Matriz de confusão:")
print(conf_matrix)

# Filtrar dados onde houve carregamento
charging_data = merged_df[merged_df['Carregar'] == 1]

# Features para o modelo de regressão
X_reg = charging_data[['Weekday', 'Days_since_last_charge']].values
y_reg = charging_data['Total Energy (kWh)'].values

# Treinar o modelo de regressão linear
reg_model = GradientBoostingRegressor()
reg_model.fit(X_reg, y_reg)

# Fazer previsões (usando X_test completo)
energy_predictions = np.zeros(len(X_test))  # Inicializar com zeros
charging_indices = [i for i, pred in enumerate(predictions) if pred == 1]

if len(charging_indices) > 0:
    X_test_charging = X_test[charging_indices]
    energy_predictions_charging = reg_model.predict(X_test_charging)
    for idx, val in zip(charging_indices, energy_predictions_charging):
        energy_predictions[idx] = val

# Consolidação dos resultados com datas corretas
test_indices = X_test[:, 0].astype(int)
result_df = pd.DataFrame({
    'Date': merged_df.iloc[test_indices].index,
    'Weekday': X_test[:, 0],
    'Days_since_last_charge': X_test[:, 1],
    'Carregar (Previsão)': predictions,
    'Total Energy (Previsão)': energy_predictions
})

result_df['Carregar (Previsão)'] = result_df['Carregar (Previsão)'].map({0: 'Não Carregar', 1: 'Carregar'})

print("Previsões finais:")
print(result_df)

Acurácia do modelo: 1.0
Matriz de confusão:
[[55  0]
 [ 0 22]]
Previsões finais:
    Date  Weekday  Days_since_last_charge Carregar (Previsão)  \
0      2        2                       0            Carregar   
1      5        5                       0            Carregar   
2      6        6                       8        Não Carregar   
3      1        1                       7        Não Carregar   
4      3        3                       0            Carregar   
..   ...      ...                     ...                 ...   
72     6        6                       2        Não Carregar   
73     2        2                       0            Carregar   
74     6        6                       1        Não Carregar   
75     2        2                       0            Carregar   
76     2        2                       5        Não Carregar   

    Total Energy (Previsão)  
0                 10.120809  
1                  9.601473  
2                  0.000000  
3                 